In [1]:
import sys, os
import re, nltk
from nltk import pos_tag
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn, gensim
from sklearn.decomposition import PCA
from gensim.corpora import Dictionary
sys.path.append(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/src/')
sys.path.append(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/src/')
os.chdir(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/data/')
from utils import preprocess_for_bow
from models.lda import LDAwrappers
from models.hdp import HDPwrapper
from models.gsdmm import MovieGroupProcessWrapper

In [7]:
encodings_to_try = ['utf-8', 'iso-8859-1', 'windows-1252', 'utf-16', 'latin-1']
for encoding in encodings_to_try:
    try:
        with open('/Users/romainbourgeois/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/data/glove/glove.6B/glove.6B.300d.txt', 'r', encoding=encoding) as file:
            content = file.read()
            print(encoding)
        break  # Exit the loop if successful
    except UnicodeDecodeError:
        continue

iso-8859-1


In [15]:
try:
    with open('/Users/romainbourgeois/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/data/glove/glove.6B/glove.6B.300d.txt', 'r', encoding='utf-8') as file:
        content = file.read()
except UnicodeDecodeError:
    with open('/Users/romainbourgeois/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/data/glove/glove.6B/glove.6B.300d.txt', 'r', encoding='iso-8859-1') as file:
        content = file.read()
        with open('/Users/romainbourgeois/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/dataglove.6B.300d.txt', 'w', encoding='utf-8') as file:
            file.write(content)
        

In [19]:
from gensim.scripts.glove2word2vec import get_glove_info
get_glove_info('/Users/romainbourgeois/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/dataglove.6B.300d.txt')

(1652021, 4)

In [16]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

glove_file = datapath('/Users/romainbourgeois/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/dataglove.6B.300d.txt' )
tmp_file = get_tmpfile("test_word2vec.txt")

_ = glove2word2vec(glove_file, tmp_file)

model = KeyedVectors.load_word2vec_format(tmp_file)

/var/folders/5_/pzjx0g9x0mgc072m4pznhqhr0000gn/T/ipykernel_53982/1344101836.py:8: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  _ = glove2word2vec(glove_file, tmp_file)


ValueError: could not convert string to float: 'http://www.berkshirehathaway.com\x94\x8c\x08kolchuha\x94\x8c\x07euro178\x94\x8c\x05kehrv\x94\x8c\x07plaisir\x94\x8c'

In [2]:
data = preprocess_for_bow('data.csv')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/romainbourgeois/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/romainbourgeois/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/romainbourgeois/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
lda=LDAwrappers(data['corpus'], data['dictionary'], 'LdaModelGensim', 10)

In [13]:
coherence_model = gensim.models.coherencemodel.CoherenceModel(topics=[x['words'] for x in lda.topics()], texts=data['tokenized_data'], dictionary=data['dictionary'], 
                                                                            coherence='c_uci', corpus=data['corpus'])
coherence_model.get_coherence_per_topic()


In [4]:
lda.coherence(data['tokenized_data'], ['c_we'])

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte

In [4]:
lda.predict_rawtext('Hello, this an example that jopefully will work.')


[(1, 0.93611103)]

In [5]:
lda.predict_corpus('data.csv')

[[(7, 0.9898338)],
 [(8, 0.9901112)],
 [(7, 0.9931935)],
 [(1, 0.99147654)],
 [(1, 0.24321446), (2, 0.5251843), (9, 0.22358239)],
 [(2, 0.9907279)],
 [(9, 0.98740923)],
 [(1, 0.98927146)],
 [(2, 0.989269)],
 [(0, 0.99026084)],
 [(8, 0.98596793)],
 [(3, 0.9897555)],
 [(9, 0.99077463)],
 [(9, 0.99038625)],
 [(6, 0.4252376), (9, 0.5671579)],
 [(2, 0.7911346), (6, 0.20002472)],
 [(3, 0.17728758), (6, 0.6797305), (9, 0.13435884)],
 [(2, 0.5949497), (9, 0.3980726)],
 [(0, 0.9902483)],
 [(4, 0.9907467)],
 [(6, 0.9920834)],
 [(8, 0.9910169)],
 [(3, 0.9906765)],
 [(0, 0.83450943), (8, 0.1570345)],
 [(8, 0.9883993)],
 [(3, 0.9923825)],
 [(3, 0.992912)],
 [(7, 0.9911335)],
 [(8, 0.98959774)],
 [(8, 0.9883951)],
 [(6, 0.9883943)],
 [(9, 0.9856481)],
 [(9, 0.991221)],
 [(0, 0.65454596), (2, 0.33836)],
 [(9, 0.9898971)],
 [(8, 0.98592293)],
 [(6, 0.9895951)],
 [(7, 0.99184185)],
 [(4, 0.99150836)],
 [(8, 0.52286845), (9, 0.4674289)],
 [(8, 0.9913929)],
 [(9, 0.98935246)],
 [(0, 0.38387462), (2, 0.33

In [6]:
lda.get_document_topics(data['corpus'][:4], )

[[(7, 0.9898338)], [(8, 0.9901112)], [(7, 0.9931935)], [(1, 0.99147654)]]

In [7]:
lda.get_indexes_per_topics(data['corpus'][:4], 0.5, data['ids'])

defaultdict(list,
            {'7': ['0011ce8df782ef32bfebad2940dcf9ac23ffb5ab4418254dc2af651',
              '0043afdfd1c84e9ec604e74358345f8ce57636eac1d1a6da69b0a74'],
             '8': ['003bf2931fdad53154b0563ab0c984915eef8399896ef7a03a95005'],
             '1': ['0062e48fa1b5e416c658639fcf2db76a171dfe088879a9e4b9e25cf']})

In [8]:
lda.topics()

[{'words': ['show',
   'covid',
   'vaccine',
   'video',
   'woman',
   'make',
   'claim',
   'year',
   'get',
   'vote'],
  'weights': [0.009438689,
   0.0070719426,
   0.0068360292,
   0.0050086114,
   0.0048475317,
   0.004589385,
   0.004395172,
   0.004354501,
   0.0041204677,
   0.0039733886]},
 {'words': ['show',
   'trump',
   'covid',
   'people',
   'president',
   'take',
   'claim',
   'video',
   'know',
   'mask'],
  'weights': [0.010356423,
   0.009936917,
   0.008810593,
   0.0059257327,
   0.005080433,
   0.0045280443,
   0.0044421037,
   0.0042590243,
   0.004021385,
   0.004000873]},
 {'words': ['covid',
   'video',
   'president',
   'show',
   'people',
   'make',
   'trump',
   'get',
   'biden',
   'state'],
  'weights': [0.01147461,
   0.0074154134,
   0.007120899,
   0.0060850554,
   0.006072803,
   0.0058318726,
   0.005513439,
   0.004801123,
   0.0042759515,
   0.0039903894]},
 {'words': ['covid',
   'show',
   'vaccine',
   'people',
   'say',
   'image'

In [9]:
lda.topic(2)

{'words': ['covid',
  'video',
  'president',
  'show',
  'people',
  'make',
  'trump',
  'get',
  'biden',
  'state'],
 'weights': [0.01147461,
  0.0074154134,
  0.007120899,
  0.0060850554,
  0.006072803,
  0.0058318726,
  0.005513439,
  0.004801123,
  0.0042759515,
  0.0039903894]}

In [10]:
from gensim.utils import grouper
c=0
chunks=[]
for i in grouper(data['corpus'], 256):
    c+=1
    chunks.append(i)
c

118

In [11]:
len(chunks[-1])

239

## HDP

In [12]:
hdp=HDPwrapper(data['corpus'], data['dictionary'])

In [13]:
hdp.predict_rawtext('Hello, this an example that jopefully will work.')

[[(0, 3.0594660296730942),
  (1, 0.03304459787751026),
  (2, 0.03142033337072021),
  (3, 0.023679158747523413),
  (4, 0.02122448638083275),
  (5, 0.019423043674836528),
  (6, 0.016146812637468522),
  (7, 0.01537470659352086),
  (8, 0.01534149075268777),
  (9, 0.012794995525704959),
  (10, 0.011674244946999857),
  (11, 0.012013557398531752),
  (12, 0.011534254389384728),
  (13, 0.010805797847376423),
  (14, 0.011271434998801),
  (15, 0.010092367426529542),
  (16, 0.01072255761539398),
  (17, 0.010279176763852328),
  (18, 0.010090561661992835),
  (19, 0.009763043629944023),
  (20, 0.009694549812908121),
  (21, 0.009556353838337313),
  (22, 0.009346525962957208),
  (23, 0.009394271935714633),
  (24, 0.00879673424182707),
  (25, 0.008405928089040237),
  (26, 0.008237685351237612),
  (27, 0.008483693467288106),
  (28, 0.008458389159545361),
  (29, 0.008519423529693945),
  (30, 0.008075022109234673),
  (31, 0.007652544078202342),
  (32, 0.007727550154863335),
  (33, 0.007939634170414286),
  

In [14]:
hdp.get_document_topics(data['corpus'][:4])

[[(0, 8.235879688496306), (3, 11.847265499924296)],
 [(0, 17.442568638203618), (1, 2.649941989346981)],
 [(0, 24.564408433510586), (59, 5.5007318052680985)],
 [(0, 19.024599098910777), (4, 5.056091417143135)]]

In [15]:
hdp.get_indexes_per_topics(data['corpus'][:4], 0.5, data['ids'])

defaultdict(list,
            {'0': ['0011ce8df782ef32bfebad2940dcf9ac23ffb5ab4418254dc2af651',
              '003bf2931fdad53154b0563ab0c984915eef8399896ef7a03a95005',
              '0043afdfd1c84e9ec604e74358345f8ce57636eac1d1a6da69b0a74',
              '0062e48fa1b5e416c658639fcf2db76a171dfe088879a9e4b9e25cf'],
             '3': ['0011ce8df782ef32bfebad2940dcf9ac23ffb5ab4418254dc2af651'],
             '1': ['003bf2931fdad53154b0563ab0c984915eef8399896ef7a03a95005'],
             '59': ['0043afdfd1c84e9ec604e74358345f8ce57636eac1d1a6da69b0a74'],
             '4': ['0062e48fa1b5e416c658639fcf2db76a171dfe088879a9e4b9e25cf']})

In [16]:
hdp.topics()

[{'words': ['show',
   'covid',
   'trump',
   'people',
   'president',
   'say',
   'video',
   'vaccine',
   'get',
   'state'],
  'weights': [0.008936396938241454,
   0.007514343919423209,
   0.007507710451853259,
   0.006621782383208056,
   0.006304467897469928,
   0.00590746392504635,
   0.0047547274721455556,
   0.004198539586226195,
   0.004081801639807703,
   0.0036779050481021754]},
 {'words': ['tweet',
   'trump',
   'people',
   'covid',
   'get',
   'show',
   'say',
   'president',
   'birdwatch',
   'try'],
  'weights': [0.005666724889535671,
   0.005378344251979359,
   0.004964610868684574,
   0.004884270346935206,
   0.004625968656439508,
   0.004170698880888041,
   0.003995204546671713,
   0.0035707778010281807,
   0.0033889539688230106,
   0.0032777768833476277]},
 {'words': ['show',
   'covid',
   'trump',
   'people',
   'president',
   'tweet',
   'say',
   'vaccine',
   'get',
   'video'],
  'weights': [0.006442068278825684,
   0.005601837375057794,
   0.00455966

In [17]:
hdp.topic(3)

{'words': ['covid',
  'people',
  'show',
  'trump',
  'vaccine',
  'say',
  'get',
  'president',
  'world',
  'video'],
 'weights': [0.004850009824902171,
  0.003968560376719306,
  0.003922237020432138,
  0.0038011237086554525,
  0.0037416704285996995,
  0.0028654348553800676,
  0.0024476829940468185,
  0.0024037394586328307,
  0.0022838288403538172,
  0.002237898311738561]}

## gsdmm

In [18]:
gsdmm=MovieGroupProcessWrapper(data['corpus'], data['dictionary'])

In stage 0: transferred 23540 clusters with 8 clusters populated
In stage 1: transferred 15092 clusters with 8 clusters populated
In stage 2: transferred 10562 clusters with 8 clusters populated
In stage 3: transferred 7856 clusters with 8 clusters populated
In stage 4: transferred 6327 clusters with 8 clusters populated
In stage 5: transferred 5548 clusters with 8 clusters populated
In stage 6: transferred 5112 clusters with 8 clusters populated
In stage 7: transferred 4700 clusters with 8 clusters populated
In stage 8: transferred 4444 clusters with 8 clusters populated
In stage 9: transferred 4287 clusters with 8 clusters populated
In stage 10: transferred 4118 clusters with 8 clusters populated
In stage 11: transferred 4045 clusters with 8 clusters populated
In stage 12: transferred 4109 clusters with 8 clusters populated
In stage 13: transferred 4066 clusters with 8 clusters populated
In stage 14: transferred 4097 clusters with 8 clusters populated
In stage 15: transferred 4068 cl

In [19]:
gsdmm.get_document_topics(data['corpus'][:4])

[[(1, 0.9999999999560342)],
 [(1, 0.9999999992819577)],
 [(5, 0.9999999998392824)],
 [(5, 0.9999999269869997)]]

In [20]:
gsdmm.get_indexes_per_topics(data['corpus'][:4], 0.5, data['ids'])

defaultdict(list,
            {'1': ['0011ce8df782ef32bfebad2940dcf9ac23ffb5ab4418254dc2af651',
              '003bf2931fdad53154b0563ab0c984915eef8399896ef7a03a95005'],
             '5': ['0043afdfd1c84e9ec604e74358345f8ce57636eac1d1a6da69b0a74',
              '0062e48fa1b5e416c658639fcf2db76a171dfe088879a9e4b9e25cf']})

In [21]:
gsdmm.topics()

[{'words': [(118, 1),
   (935, 1),
   (5132, 1),
   (5993, 1),
   (1650, 1),
   (3292, 1),
   (3249, 1),
   (2813, 1),
   (1734, 1),
   (244, 1)],
  'weights': [0.041967319957171456,
   0.02388156975932219,
   0.01457101624691588,
   0.011545086355383828,
   0.009310553512406313,
   0.0076346538801731765,
   0.007122573436990829,
   0.006587216610027466,
   0.006331176388436293,
   0.006028583399283087]},
 {'words': [(69, 1),
   (28, 1),
   (85, 1),
   (156, 1),
   (120, 1),
   (30, 1),
   (219, 1),
   (64, 1),
   (254, 1),
   (165, 1)],
  'weights': [0.016428571428571428,
   0.010133928571428572,
   0.007455357142857143,
   0.007321428571428572,
   0.005699404761904762,
   0.0056696428571428575,
   0.005625,
   0.005461309523809524,
   0.005074404761904762,
   0.004955357142857142]},
 {'words': [(69, 1),
   (85, 1),
   (151, 1),
   (28, 1),
   (111, 1),
   (11, 1),
   (30, 1),
   (52, 1),
   (118, 1),
   (224, 1)],
  'weights': [0.028827536751523844,
   0.020031074459184894,
   0.0118

In [22]:
gsdmm.topic(3)

{'words': [(2462, 2),
  (1455, 1),
  (148, 1),
  (5377, 1),
  (19024, 1),
  (97, 1),
  (248, 1),
  (126, 1),
  (4295, 1),
  (14495, 1)],
 'weights': [0.05314685314685315,
  0.05314685314685315,
  0.053069153069153066,
  0.053069153069153066,
  0.049339549339549336,
  0.04871794871794872,
  0.04265734265734266,
  0.04257964257964258,
  0.04257964257964258,
  0.04257964257964258]}

In [23]:
gsdmm.predict_rawtext('Hello, this an example that jopefully will work.')

[[(0, 0.13253150124549107),
  (1, 0.048140648184350335),
  (2, 0.11775312001543077),
  (3, 0.20814293935456282),
  (4, 0.21115197440660088),
  (5, 0.0609413350431685),
  (6, 0.10862166033090996),
  (7, 0.11271682141948558)],
 [(0, 0.13253150124549107),
  (1, 0.048140648184350335),
  (2, 0.11775312001543077),
  (3, 0.20814293935456282),
  (4, 0.21115197440660088),
  (5, 0.0609413350431685),
  (6, 0.10862166033090996),
  (7, 0.11271682141948558)],
 [(0, 0.13253150124549107),
  (1, 0.048140648184350335),
  (2, 0.11775312001543077),
  (3, 0.20814293935456282),
  (4, 0.21115197440660088),
  (5, 0.0609413350431685),
  (6, 0.10862166033090996),
  (7, 0.11271682141948558)]]

In [6]:
lda.coherence(data['tokenized_data'],['c_we'])

KeyboardInterrupt: 

In [4]:
lda.coherence(data['tokenized_data'], ['c_npmi'])

defaultdict(dict,
            {'c_npmi': {'c_npmi_per_topic': [-0.025088489318943325,
               -0.06582192041397027,
               -0.0650665629773811,
               -0.12808257999064115,
               0.008416846828953517,
               -0.028707520323148956,
               0.011540398760295011,
               -0.19800097930868973,
               -0.03357265962159351,
               0.6985826391892829],
              'c_npmi': 0.01741991728241633,
              'c_npmi_std': 0.23493479693989563}})

In [8]:
lda.coherence(data['tokenized_data'], ['c_uci'])

Coherence_Measure(seg=<function s_one_one at 0x177e3bb80>, prob=<function p_boolean_sliding_window at 0x177e3bdc0>, conf=<function log_ratio_measure at 0x177e96ca0>, aggr=<function arithmetic_mean at 0x177e9b3a0>)


In [ ]:
lda.coherence(data['tokenized_data'], ['u_mass'])

In [2]:
lda.coherence(data['tokenized_data'], ['c_v', 'c_npmi', 'c_uci', 'u_mass'])

NameError: name 'lda' is not defined

In [ ]:
lda.coherence(data['tokenized_data'])

In [ ]:
hdp.coherence(data['tokenized_data'])

TypeError: 'method' object is not iterable

In [1]:
gsdmm.coherence(data['tokenized_data'])

NameError: name 'gsdmm' is not defined

In [ ]:
data.keys()

dict_keys(['text', 'ids', 'tokenized_data', 'dictionary', 'corpus'])

In [ ]:
from utils import preprocess_for_bow, preprocess
preproc_params = {'keep_unicodes': {'keep': True, 'min_count_in_corpus': 2}, 'strip_brackets': False, 
                                          'add_adj_nn_pairs': True, 'verbs': True, 'adjectives': False}
data2=preprocess('Hello, this an example that jopefully will work.',  preproc_params['strip_brackets'], preproc_params['keep_unicodes']['keep'], 
                preproc_params['add_adj_nn_pairs'],  preproc_params['verbs'], 
                preproc_params['adjectives']) 
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
tokenized_data = [token.strip() for token in tokenizer.tokenize(data2)]
tokenized_data

In [ ]:
data['corpus'][0]

In [ ]:
hdp.get_document_topics(data['corpus'][0])

In [ ]:
data['dictionary'].doc2bow(tokenized_data)

[(15, 1), (94, 1), (886, 1)]

In [ ]:
hdp.get_document_topics(hdp.id2word.doc2bow(tokenized_data), None) 

TypeError: '>' not supported between instances of 'float' and 'NoneType'

In [ ]:
hdp.get_document_topics(data['corpus'][0])

[[(0, 7.106150002153514), (11, 12.951359723971796)]]

In [ ]:
data['corpus'][0]

[(0, 1),
 (1, 2),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 2),
 (15, 1),
 (16, 1),
 (17, 1)]